In [1]:
import os
import time

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_fireworks import ChatFireworks
from langchain_google_vertexai import ChatVertexAI
from langchain_openai import ChatOpenAI
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

# initialize env vars - load credentials
load_dotenv()

# initialize graph db:
graph = Neo4jGraph()

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0,
    max_tokens=2000,
    )

# initialize graph transformer:
llm_transformer = LLMGraphTransformer(
    llm=llm,
)

In [5]:
def load_policies(
    file: str = "../../data/clean/policy_chunks.csv",
    start_index: int = 0,
    limit: int = 100,
):
    # load dataframe:
    df = pd.read_csv(file)[start_index:start_index + limit]
    print(f"Loaded policies: {df.shape}")
    df["Content"] = df.apply(lambda row: f"{row['Header 1']}\n{row['Header 2']}\n{row['Concat']}".replace("\n\n", "\n").replace("\n\n", "\n"), axis=1)
    return df

In [6]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def convert_to_graph_documents(df: pd.DataFrame, page_content_column: str = "Content", llm_transformer: LLMGraphTransformer = None):
    # graph_documents = llm_transformer.convert_to_graph_documents(documents)
    loader = DataFrameLoader(df, page_content_column=page_content_column)
    documents = loader.load()
    # return  llm_transformer.convert_to_graph_documents(documents)
    graph_documents = []
    for doc in documents:
        try:
            # fireworks doesnt always return valid json
            graph_documents += llm_transformer.convert_to_graph_documents([doc])
        except:
            graph_documents += [None]
    return graph_documents

In [7]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0,
    max_tokens=2000,
)

# initialize graph transformer:
llm_transformer = LLMGraphTransformer(
    llm=llm,
)

policies_df = load_policies(start_index=0, limit=1)
gd = convert_to_graph_documents(policies_df, llm_transformer=llm_transformer)
print(len(gd[0].nodes), gd[0].nodes)
print(len(gd[0].relationships), gd[0].relationships)

Loaded policies: (1, 4)
19 [Node(id='Carry-On Bags', type='Item'), Node(id='Personal Item', type='Item'), Node(id='Diaper Bags', type='Item'), Node(id='Breast Pump', type='Item'), Node(id='Cooler Of Breast Milk', type='Item'), Node(id='Child Safety Seats', type='Item'), Node(id='Strollers', type='Item'), Node(id='Medical Or Mobility Devices', type='Item'), Node(id='Carry-On Requirements', type='Requirement'), Node(id='Soft-Sided Garment Bags', type='Item'), Node(id='Musical Instruments', type='Item'), Node(id='Special Items And Sports Equipment', type='Item'), Node(id='Pets', type='Item'), Node(id='Valet Bags', type='Item'), Node(id='Batteries', type='Item'), Node(id='Electronic Cigarettes', type='Item'), Node(id='Restricted Items', type='Item'), Node(id='Liquids', type='Item'), Node(id='Tsa Liquids Rule', type='Rule')]
18 [Relationship(source=Node(id='Personal Item', type='Item'), target=Node(id='Carry-On Bags', type='Item'), type='BELONGS_TO'), Relationship(source=Node(id='Diaper Bag

In [8]:
llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    temperature=0,
    max_tokens=2000,
)

# initialize graph transformer:
llm_transformer = LLMGraphTransformer(
    llm=llm,
)

policies_df = load_policies(start_index=0, limit=1)
gd = convert_to_graph_documents(policies_df[0:1], llm_transformer=llm_transformer)
print(len(gd[0].nodes), gd[0].nodes)
print(len(gd[0].relationships), gd[0].relationships)

Loaded policies: (1, 4)
15 [Node(id='Personal Item', type='Item'), Node(id='Carry-On Bag', type='Item'), Node(id='Diaper Bags', type='Item'), Node(id='Breast Pump', type='Item'), Node(id='Small, Soft-Sided Cooler Of Breast Milk', type='Item'), Node(id='Child Safety Seats', type='Item'), Node(id='Strollers', type='Item'), Node(id='Medical Or Mobility Devices', type='Item'), Node(id='Soft-Sided Garment Bags', type='Item'), Node(id='Musical Instruments', type='Item'), Node(id='Pet Kennel Or Container', type='Item'), Node(id='Valet Bags', type='Item'), Node(id='American Eagle', type='Airline'), Node(id='Embraer Erj-175', type='Aircraft'), Node(id='Tsa', type='Organization')]
10 [Relationship(source=Node(id='Diaper Bags', type='Item'), target=Node(id='Personal Item', type='Item'), type='EXCLUDED_FROM'), Relationship(source=Node(id='Breast Pump', type='Item'), target=Node(id='Personal Item', type='Item'), type='EXCLUDED_FROM'), Relationship(source=Node(id='Small, Soft-Sided Cooler Of Breast 

In [9]:
document = """


1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.




1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.




1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
Jan 09, 2024 published  
Refunds of EVA/ UNI Air Ticket for Komatsu Routes due to Noto Peninsula Earthquakes in Japan","Ticket Changes/Refunds due to Flight Irregularities
Changes/Refunds of EVA/UNI Air Tickets affected by Aircraft Change
1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.

1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.

1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
Jan 09, 2024 published  
"""

document_df = pd.DataFrame([{"Content": document}])

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0,
    max_tokens=2000,
)

# initialize graph transformer:
llm_transformer = LLMGraphTransformer(
    llm=llm,
)

gd = convert_to_graph_documents(document_df, llm_transformer=llm_transformer)
print(len(gd[0].nodes), gd[0].nodes)
print(len(gd[0].relationships), gd[0].relationships)

27 [Node(id='Passengers', type='Passenger'), Node(id='Br (695)', type='Ticket'), Node(id='B7(525)', type='Ticket'), Node(id='Premium Economy', type='Booking'), Node(id='Aircraft Change', type='Event'), Node(id='Economy Booking Class', type='Booking class'), Node(id='Flight', type='Route'), Node(id='Fare/Tax Difference', type='Financial transaction'), Node(id='Rebooking', type='Action'), Node(id='Refund', type='Action'), Node(id='Reissue Fee', type='Fee'), Node(id='Ticket Validity', type='Validity'), Node(id='Travel Agent', type='Agent'), Node(id='Flight Irregularities', type='Event'), Node(id='Award Ticket', type='Ticket'), Node(id='Upgrade', type='Action'), Node(id='Mileages', type='Measurement'), Node(id='Service Fee', type='Fee'), Node(id='Expired Miles', type='Measurement'), Node(id='Green Card', type='Membership'), Node(id='Diamond Card', type='Membership'), Node(id='Gold Card', type='Membership'), Node(id='Silver Card', type='Membership'), Node(id='Business Class', type='Booking 

In [10]:
llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    temperature=0,
    max_tokens=2000,
)

# initialize graph transformer:
llm_transformer = LLMGraphTransformer(
    llm=llm,
)

gd = convert_to_graph_documents(document_df, llm_transformer=llm_transformer)
print(len(gd[0].nodes), gd[0].nodes)
print(len(gd[0].relationships), gd[0].relationships)